In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = open('/content/HP1_paragraph.txt').read().lower()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def word_count(paragraph):
  word_count = 0
  for i in range(len(paragraph)):
      if paragraph[i] == ' ' or paragraph[i] == '\n':
        word_count += 1
  return word_count

In [ ]:
chars = set(data)
data_size, char_size = len(data), len(chars)
print(f'Data size: {data_size}, Char Size: {char_size}')
print(data[:10])
word_count_data = word_count(data)
print(f'word_count_data = {word_count_data}')

Data size: 1545, Char Size: 31
harry pott
word_count_data = 283


In [ ]:
split_index = int(0.8 * len(data))

while data[split_index] != ' ' and data[split_index] != '\n':
  split_index += 1

train_data = data[:split_index]
test_data = data[split_index:]

print(f'Training data size: {len(train_data)}, Training char size: {len(set(train_data))}')
print(f'Test data size: {len(test_data)}, Test char size: {len(set(test_data))}')

word_count_train = word_count(train_data)
word_count_test = word_count(test_data)
print(f'word_count_train = {word_count_train}')
print(f'word_count_test = {word_count_test}')

print(f'train_data = {train_data}')
print(f'test_data = {test_data}')

Training data size: 1236, Training char size: 31
Test data size: 309, Test char size: 28
word_count_train = 226
word_count_test = 57
train_data = harry potter and the sorcerer's stone 

chapter one 

the boy who lived 

mr. and mrs. dursley, of number four, privet drive, were proud to say that they were perfectly normal, thank you very much. they were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense. 

mr. dursley was the director of a firm called grunnings, which made drills. he was a big, beefy man with hardly any neck, although he did have a very large mustache. mrs. dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. the dursleys had a small son called dudley and in their opinion there was no finer boy anywhere. 

the dursleys had everything they wanted, but they also had

In [ ]:
# Sample paragraph
paragraph = train_data

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([paragraph])
total_words = len(tokenizer.word_index) + 1

# Create input sequences using list of tokens
input_sequences = []
for line in paragraph.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences to ensure uniform length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and label
X, y = input_sequences[:,:-1],input_sequences[:,-1]

# Define the LSTM model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit model
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
7/7 [==============================] - 4s 118ms/step - loss: 4.9195 - accuracy: 0.0188
Epoch 2/100
7/7 [==============================] - 1s 106ms/step - loss: 4.8998 - accuracy: 0.0469
Epoch 3/100
7/7 [==============================] - 0s 69ms/step - loss: 4.8385 - accuracy: 0.0516
Epoch 4/100
7/7 [==============================] - 0s 69ms/step - loss: 4.7608 - accuracy: 0.0329
Epoch 5/100
7/7 [==============================] - 0s 68ms/step - loss: 4.7168 - accuracy: 0.0329
Epoch 6/100
7/7 [==============================] - 0s 70ms/step - loss: 4.6617 - accuracy: 0.0423
Epoch 7/100
7/7 [==============================] - 0s 69ms/step - loss: 4.6191 - accuracy: 0.0376
Epoch 8/100
7/7 [==============================] - 0s 70ms/step - loss: 4.5743 - accuracy: 0.0469
Epoch 9/100
7/7 [==============================] - 0s 70ms/step - loss: 4.5195 - accuracy: 0.0469
Epoch 10/100
7/7 [==============================] - 0s 70ms/step - loss: 4.4497 - accuracy: 0.0563
Epoch 11/100
7/7 

In [ ]:
# Function to generate text given a seed text
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generate text
first_word_index = 1
while test_data[first_word_index] != ' ' and test_data[first_word_index] != '\n':
  first_word_index += 1
test_paragraph = test_data[:first_word_index]
print(f'test_paragraph = {test_paragraph}')
generated_text = generate_text(test_paragraph, word_count_test, model, max_sequence_len)
print(generated_text)


test_paragraph =  the
 the dursleys had everything they wanted but they also had a secret and their greatest fear was that somebody would discover it they didn't think they could bear it if anyone found out about the potters mrs potter was mrs dursley's sister but they hadn't met for several years in fact mrs dursley pretended she didn't have a


In [ ]:
#calculate accuracy by char
print(f'len(test_data) = {len(test_data)}')
print(f'len(generated_text) = {len(generated_text)}\n')
min_len = min(len(test_data), len(generated_text))

print(f'test_data = {test_data}\n')
print(f'generated_text = {generated_text}')
print(generated_text)
match_count = 0
for i in range(min_len):
  if test_data[i] == generated_text[i]:
    match_count += 1
accuracy = match_count / min_len
print(f'accuracy = {round(accuracy * 100, 2)}%')

len(test_data) = 309
len(generated_text) = 324

test_data =  the dursleys shuddered to think what the neighbors would say if the potters arrived in the street. the dursleys knew that the potters had a small son, too, but they had never even seen him. this boy was another good reason for keeping the potters away; they didn't want dudley mixing with a child like that. 

generated_text =  the dursleys had everything they wanted but they also had a secret and their greatest fear was that somebody would discover it they didn't think they could bear it if anyone found out about the potters mrs potter was mrs dursley's sister but they hadn't met for several years in fact mrs dursley pretended she didn't have a
 the dursleys had everything they wanted but they also had a secret and their greatest fear was that somebody would discover it they didn't think they could bear it if anyone found out about the potters mrs potter was mrs dursley's sister but they hadn't met for several years in fact mr